In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import torchvision.transforms as tvt
import torch.nn.functional as F

from PIL import Image
import numpy as np
import os, glob

/Users/varun/miniforge3/envs/o4a/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [3]:
class mynet(nn.Module):
    def __init__(self):
        super(mynet, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)  # (A)
        self.conv2 = nn.Conv2d(128, 128, 3)  # (B)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128*31*31, 1000)  # (C)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        ## Uncomment the next statement and see what happens to the
        ## performance of your classifier with and without padding.
        ## Note that you will have to change the first arg in the
        ## call to Linear in line (C) above and in the line (E)
        ## shown below. After you have done this experiment, see
        ## if the statement show n below can be invoked twice with
        ## and without padding. How about three times?
        # x = self.pool(F.relu(self.conv2(x))) ## (D)
        x = x.view(-1, 128*31*31)  ## (E)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
class dataLoader(Dataset):
    def __init__(self,dataPath,classes,transform):
        self.dataPath = dataPath
        self.classes = classes
        self.transform = transform
        # image list and corresponding 
        self.imgList = []
        self.labelForImg()
        
    def __len__(self):
        length = 0
        for cls in self.classes:
            length += len(glob.glob(os.path.join(self.dataPath,cls,"*")))
        return length
    
    def __getitem__(self,idx):
        img_path = self.imgList[idx,1]
        label = torch.tensor(int(self.imgList[idx,0]), dtype=torch.uint8)
        
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        
        return image, label
    
    def labelForImg(self):
        for i, cls in enumerate(self.classes):
            imgs = glob.glob(os.path.join(self.dataPath,cls,'*'))
            if i==0:
                self.imgList = np.vstack((i*np.ones(len(imgs),dtype=int),imgs))
            else:
                self.imgList = np.hstack((self.imgList,np.vstack((i*np.ones(len(imgs),dtype=int),imgs))))
        self.imgList = self.imgList.T

In [5]:
def run_code_for_training(net, train_data_loader):
    net = net.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_data_loader):
            (inputs, labels) = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i + 1) % 500 == 0:
                print("\n[epoch:%d, batch:%5d] loss: %.3f"% (epoch + 1, i + 1, running_loss / float(500)))
                running_loss = 0.0

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
batch=8
epochs = 10
dataPath = "./hw04_coco_data/Train"
transform = tvt.Compose([tvt.ToTensor(), tvt.Normalize((0.5,0.5, 0.5), (0.5, 0.5, 0.5))])            

net = mynet()            
dt = dataLoader(dataPath,["refrigerator","airplane","giraffe","cat","elephant","dog","train","horse","boat","truck"],transform)
TrainDataLoader = torch.utils.data.DataLoader(dataset = dt, batch_size = batch, shuffle = True, num_workers = 4)
run_code_for_training(net,TrainDataLoader)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/varun/miniforge3/envs/o4a/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/varun/miniforge3/envs/o4a/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'dataLoader' on <module '__main__' (built-in)>


KeyboardInterrupt: 

# 